In [33]:
import pandas as pd
import numpy as np
from groq import Groq
from openai import OpenAI
from pyairtable import Table,Api
from pprint import pprint
import networkx as nx
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.cluster.util import cosine_distance


In [34]:
api = Api('patkECjZGaeJ3Rj2F.902a4dc39c614cbcba594a1d909db35a39dcd69f43d2bf692aaf32268c4d60d7')
tableMessages = api.table('appY85YsYveVhA1eC', 'Messages')
data=tableMessages.all()
fields = [record['fields'] for record in data]
df=pd.DataFrame(fields)

In [35]:
dff = df['other'].dropna().to_list()

In [ ]:
pprint(dff)

In [11]:
from networkx.algorithms.link_analysis.pagerank_alg import pagerank
from sentence_transformers import SentenceTransformer, util

class BERTSummarizer:

    def __init__(self, pretrained_model='all-MiniLM-L6-v2'):
        self.bert_model = SentenceTransformer(pretrained_model)

    def create_graph(self, sentences, sentence_vectors):
        G = nx.Graph()
        for i, s1 in enumerate(sentences):
            for j, s2 in enumerate(sentences):
                # do not compute similarity with itself
                if s1 != s2:
                    similarity = util.pytorch_cos_sim(sentence_vectors[i], sentence_vectors[j])
                    G.add_edge(i, j, weight=similarity)
        return G

    def summarize(self, sentences, N=2, return_list=False):
        sentence_vectors = self.bert_model.encode(sentences)
        G = self.create_graph(sentences, sentence_vectors)
        try:
            pr_scores = pagerank(G, max_iter=1000)
        except Exception as e:
            print("The PageRank algorithm failed to converge. Returning the top sentences according to their position in the text.")
            pr_scores = {i: N-i for i in range(len(sentences))}

        ordered_pr = dict(sorted(pr_scores.items(), key=lambda item: item[1], reverse=True))
        ordered_keys = list(ordered_pr.keys())
        summary_sents = []
        for ind in ordered_keys[:N]:
            summary_sents.append(sentences[ind])
        if return_list:
            return summary_sents
        else:
            return " ".join(summary_sents)

bs = BERTSummarizer()
summary = bs.summarize(dff)
print (summary)

In [38]:
phrases="\n".join(dff)
num_sent=10
model = "mixtral-8x7b-32768"
client = Groq(
        api_key='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',
    )

context = f"""You are given a list of phrases in Italian. Your task is to extract the most important objections
 and generate sentences that best describe the essence of the phrases.
Act like you are the person writing those phrases.
Make them short and understandable, capturing key aspects. 
List {num_sent} of the most important objections made.
Here are the phrases:\n\n{phrases}"""

messages = [
    {"role": "system", "content": "You are an AI language model."},
    {"role": "user", "content": context}
]

chat_completion = client.chat.completions.create(
                    messages=messages,
                    model=model,
                    max_tokens=32600,
                    temperature=1
                )
llm_response = chat_completion.choices[0].message.content
pprint(llm_response)


('1. We are not interested in working specifically on their technologies.\n'
 '2. Starting with a long message and asking if the recipient is well might '
 'not be appropriate.\n'
 '3. There are factual errors, including the name of the company, which is '
 'Airbag Studio, not Airmbag Studio.\n'
 '4. Adding "saluti saluti" at the end is too lengthy and informal.\n'
 '5. It\'s better to use "digital strategy" instead of "strategies" in '
 'Italian.\n'
 '6. There is no need to mention a commitment to technology for a '
 'technology-focused company.\n'
 '7. The message should focus on the company being contacted.\n'
 '8. The message is too lengthy and contains redundant information.\n'
 '9. Using an article before the company name, Airbag Studio, is incorrect.\n'
 '10. The innovative approach to programming may not make sense in this '
 'context.\n'
 '\n'
 'Additional objections:\n'
 '\n'
 '11. The message should not include unnecessary elements like "ci ha '
 'incuriosito" or "spero tu s